## Récupérations des données
Lecture des fichiers contenant les données et renommage des colonnes afin d'obtenir une homogénéisation pour la suite.

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

dfs = []

# Récupèration des bases de données sous forme de dataframe.
dfs.append(pd.read_csv('/kaggle/input/world-happiness/2015.csv'))
dfs.append(pd.read_csv('/kaggle/input/world-happiness/2016.csv'))
dfs.append(pd.read_csv('/kaggle/input/world-happiness/2017.csv'))
           
# Renommage des colonnes
dfs[2].rename(columns={'Economy..GDP.per.Capita.': 'Economy', 'Health..Life.Expectancy.': 'Health','Trust..Government.Corruption.': 'Trust'}, inplace=True)
for df in dfs[0:2]:
    df.rename(columns={'Economy (GDP per Capita)': 'Economy', 'Health (Life Expectancy)': 'Health','Trust (Government Corruption)': 'Trust'}, inplace=True)
    

## Evolution des critères forts par année

In [ ]:
dfMax = []

for df in dfs:
    # Pour chaque pays, on récupère la valeur la plus élevée ainsi que la catégorie correspondante.
    df['Max value'] = df[["Family", "Generosity", "Economy", "Health", "Freedom", "Trust"]].max(axis = 1)
    df['Max category'] = df[["Family", "Generosity", "Economy", "Health", "Freedom", "Trust"]].idxmax(axis = 1)
    # Ensuite, on compte le nombre d'apparitions de chaque catégorie.
    dfMax.append(df.groupby("Max category").size().reset_index(name="Max number"))

# Mise sous forme de dictionnaire des colonnes "Catégorie", "Nombre d'apparitions" pour chaque année.
df2015 = dict(zip(dfMax[0]['Max category'],dfMax[0]['Max number']))
df2016 = dict(zip(dfMax[1]['Max category'],dfMax[1]['Max number']))
df2017 = dict(zip(dfMax[2]['Max category'],dfMax[2]['Max number']))

# Création d'une nouvelle dataframe afin d'afficher toutes les années sur un même graphe.
dfbar = pd.DataFrame.from_dict({"2015": df2015, "2016": df2016, "2017": df2017}, orient='index')
dfbar['Years'] = dfbar.index
# Réarengement avec création de nouvelles colonnes.
dfbar = pd.melt(dfbar, id_vars="Years", var_name="Categories", value_name="Count")

# Affichage du graphe
graph = sns.catplot(x="Years", y="Count", hue="Categories", data=dfbar, kind="bar")
graph.fig.set_size_inches(15,8)


Le graphe obtenu permet de voir l'évolution, par année, des critères qui ont l'influence la plus positive dans la note du bonheur.
L'ordonnée correspond au nombre de fois pour lesquelles le critère est apparu comme ayant la valeur la plus forte. Ainsi, en 2015, 2016 et 2017, ce sont l'économie et la famille qui ressortent. Pour obtenir les notes dans ces catégories, le sondage initial se base sur le PIB pour l'économie et sur la question "Pouvez-vous compter sur un proche ou des amis lorsque vous en avez besoin ?" pour la famille.
<br>Cela signifie donc :
* soit que certaines, voire, la majorité des populations sont développées économiquement et/ou sont épanouies dans leurs relations personnelles, 
* soit que, malgré un faible développement économique du pays et/ou un non épanouissement dans les relations personnelles, ce sont ces critères qui prévalent.

Il se peut donc qu'il faille d'abord un développement économique assez important et/ou une stabilité dans les relations, pour que les autres domaines (qui sont les autres critères) se développent, et qu'une population se sente heureuse. <br>Les variations d'année en année entre les 2 critères majeurs peuvent provenir du fait que les valeurs maximales entre chaque catégorie sont très proches.

## Evolution des critères faibles par année

In [ ]:
dfMin = []
for df in dfs:
    # Pour chaque pays, on récupère la valeur la moins élevée ainsi que la catégorie correspondante.
    df['Min value'] = df[["Family", "Generosity", "Economy", "Health", "Freedom", "Trust"]].min(axis = 1)
    df['Min category'] = df[["Family", "Generosity", "Economy", "Health", "Freedom", "Trust"]].idxmin(axis = 1)
    # Ensuite, on compte le nombre d'apparitions de chaque catégorie.
    dfMin.append(df.groupby("Min category").size().reset_index(name="Max number"))

# Mise sous forme de dictionnaire des colonnes "Catégorie", "Nombre d'apparitions" pour chaque année.
df2015 = dict(zip(dfMin[0]['Min category'],dfMin[0]['Max number']))
df2016 = dict(zip(dfMin[1]['Min category'],dfMin[1]['Max number']))
df2017 = dict(zip(dfMin[2]['Min category'],dfMin[2]['Max number']))

# Création d'une nouvelle dataframe afin d'afficher toutes les années sur un même graphe.
dfbar = pd.DataFrame.from_dict({"2015": df2015, "2016": df2016, "2017": df2017}, orient='index')
dfbar['Years'] = dfbar.index
# Réarengement avec création de nouvelles colonnes.
dfbar = pd.melt(dfbar, id_vars="Years", var_name="Categories", value_name="Count")

# Affichage du graphe
graph = sns.catplot(x="Years", y="Count", hue="Categories", data=dfbar, kind="bar")
graph.fig.set_size_inches(16,8)

Le graphe obtenu permet de voir l'évolution, par année, des critères qui ont l'influence la moins positive dans la note du bonheur.
L'ordonnée correspond au nombre de fois pour lesquelles le critère est apparu comme ayant la valeur la moins forte. Ainsi, en 2015, 2016 et 2017, ce sont la générosité et la perception de la corruption qui ressortent. Pour obtenir les notes dans ces catégories, le sondage se base sur 2 questions : "Avez-vous donné de l'argent à une association caritative le mois dernier ?" pour la générosité et "Pensez-vous que le gouvernement et/ou les entreprises sont corrompus ?" pour la perception de la corruption. <br>Cela signifie donc que :
* soit certaines, voire la majorité, des populations sont peu généreuses et/ou n'accordent pas leur confiance envers le gouvernement et les entreprises,
* soit les populations sont généreuses et/ou ont confiance en leur gouvernement et entreprises, mais cela reste les domaines dans lesquels elles sont le moins satisfaites.

Il se peut donc que ces 2 catégories soit liées, et que plus une personne pense que son pays est corrompu, moins elle sera propice à aider les gens de son pays.
<br>Ainsi, ce serait ces domaines sur lesquels il faudrait se concentrer, si les autres domaines sont suffisamment satisfaisants, pour qu'une population se sente plus heureuse.

## Représentation des critères forts par région

In [ ]:
dfMaxRegion = []
dfMaxValue = []

for df in dfs[0:2]:
    # Pour chaque région, on compte le nombre d'apparitions de chaque catégorie étant la plus représentée.
    dfMaxRegion.append(df.groupby(["Region", "Max category"]).size().reset_index(name="count"))
    # On fait la moyenne des valeurs pour chaque colonne et récupère seulement la colonne des valeurs maximales.
    data = df.groupby(["Region", "Max category"]).mean()
    data = data["Max value"]
    dfMaxValue.append(data.to_frame())

# Affichage du graphe en barre pour l'année 2015.
graph = sns.catplot(x="Region", y="count", hue="Max category", data=dfMaxRegion[0], kind="bar")
plt.title("2015")
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)
plt.show()

# Affichage du nuage de points pour l'année 2015.
scatter = sns.scatterplot(x=dfMaxValue[0].index.get_level_values(0), y=dfMaxValue[0].index.get_level_values(1), hue="Max value", size="Max value", sizes=(30, 250), data=dfMaxValue[0])
scatter.legend(loc="center left", bbox_to_anchor=(1.1, 0.5), ncol=1)
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)

# Affichage du graphe en barre pour l'année 2016.
graph = sns.catplot(x="Region", y="count", hue="Max category", data=dfMaxRegion[1], kind="bar")
plt.title("2016")
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)
plt.show()

# Affichage du nuage de points pour l'année 2016.
scatter = sns.scatterplot(x=dfMaxValue[1].index.get_level_values(0), y=dfMaxValue[1].index.get_level_values(1), hue="Max value", size="Max value", sizes=(30, 250), data=dfMaxValue[1])
scatter.legend(loc="center left", bbox_to_anchor=(1.1, 0.5), ncol=1)
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)

Les graphes obtenus permettent de voir la répartition, selon chaque grande région mondiale, des critères qui ont l'influence la plus positive dans la note du bonheur.
Pour les graphes en barres, l'ordonnée correspond au nombre de fois pour lesquelles le critère est apparu comme ayant la valeur la plus forte. Ainsi, pour 2015 et 2016 on retrouve les résultats que l'on avait obtenus à l'échelle mondiale : l'économie et la famille sont les critères qui ressortent, avec une prévalence de la famille en 2015 et une prévalence de l'économie en 2016. 
<br><br> Pour l'année 2015 cependant, si l'on regarde plus précisément grâce au graphe en nuage de points, on observe que dans certaines régions, comme l'Afrique subsaharienne, l'Amérique latine et les Caraïbes ou l'Europe de l'Ouest, les valeurs maximales de l'économie et la famille sont très proches. Cela signifie que quelque soit le critère qui prédomine, la valeur la plus élevée reste la même. Donc il se peut que toutes les populations d'une même région aient à peu près le même ressenti du bonheur.
<br> Pour les autres régions où l'on observe une grande disparité entre les valeurs maximales, comme l'Asie du Sud et l'Asie du Sud-Est, il se peut qu'il y ait un pays dans la région qui soit bien plus satisfait dans l'un des domaines, ce qui expliquerait ces différences.
<br> Enfin, on peut constater que les régions qui ont les valeurs maximales les plus élevées sont les régions trés développées économiquement. 
<br><br> Pour l'année 2016, lorsque l'on regarde le graphe en nuage de points, on peut remarquer que les valeurs maximales se démarquent bien au niveau de l'économie pour presque toutes les régions. Cependant, si on les compare avec celles de l'année 2015, on constate que la majorité de ces valeurs ont diminuées, tout comme celles associées à la famille. Il est donc très probable que ces 2 domaines soit liés : une baisse de l'un entrênerait une baisse de l'autre. Pour les régions concernées, les populations étaient très certainement moins heureuses qu'en 2015.
<br> Les régions étant fortement développées économiquement ne sont, elles, pas impactées par ces variations.

## Représentation des critères faibles par région

In [ ]:
dfMinRegion = []
dfMinValue = []

for df in dfs[0:2]:
    # Pour chaque région, on compte le nombre d'apparitions de chaque catégorie étant la moins représentée.
    dfMinRegion.append(df.groupby(["Region", "Min category"]).size().reset_index(name="count"))
    # On fait la moyenne des valeurs pour chaque colonne et récupère seulement la colonne des valeurs minimales.
    data = df.groupby(["Region", "Min category"]).mean()
    data = data["Min value"]
    dfMinValue.append(data.to_frame())

# Affichage du graphe en barre pour l'année 2015.
graph = sns.catplot(x="Region", y="count", hue="Min category", data=dfMinRegion[0], kind="bar")
plt.title("2015")
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)
plt.show()
# Affichage du nuage de points pour l'année 2015.
scatter = sns.scatterplot(x=dfMinValue[0].index.get_level_values(0), y=dfMinValue[0].index.get_level_values(1), hue="Min value", size="Min value", sizes=(30, 250), data=dfMinValue[0])
scatter.legend(loc="center left", bbox_to_anchor=(1.1, 0.5), ncol=1)
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(35,8)

# Affichage du graphe en barre pour l'année 2016.
graph = sns.catplot(x="Region", y="count", hue="Min category", data=dfMinRegion[1], kind="bar")
plt.title("2016")
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)
plt.show()
# Affichage du nuage de points pour l'année 2016.
scatter = sns.scatterplot(x=dfMinValue[1].index.get_level_values(0), y=dfMinValue[1].index.get_level_values(1), hue="Min value", size="Min value", sizes=(30, 250), data=dfMinValue[1])
scatter.legend(loc="center left", bbox_to_anchor=(1.1, 0.5), ncol=1)
plt.xticks(rotation=45, rotation_mode="anchor", ha="right")
graph.fig.set_size_inches(20,8)

Les graphes obtenus permettent de voir la répartition, selon chaque grande région mondiale, des critères qui ont l'influence la moins positive dans la note du bonheur.
Pour les graphes en barres, l'ordonnée correspond au nombre de fois pour lesquelles le critère est apparu comme ayant la valeur la moins forte. Ainsi, pour 2015 et 2016 on retrouve les résultats que l'on avait obtenus à l'échelle mondiale : la générosité et la perception de la corruption sont les critères qui ressortent, avec une prévalence du second.
<br><br>Il y a très peu de changements entre les 2 années, bien qu'on puisse remarquer une légère augmentation de la générosité pour l'Asie du Sud-Est. Les régions étant fortement développées économiquement sont celle qui, une fois de plus, ont les valeurs les plus élevées.
Dans tous les cas ces valeurs restent assez faibles, donc ce sont ces domaines qui contribuent très peu à l'augmentation de la note du bonheur, et donc en quelque sorte font baisser cette note. C'est donc dans ces domaines qu'il faudrait se concentrer pour améliorer le bonheur de la majorité des populations.
<br><br>Cela signifierait peut-être qui le ressenti du bonheur est un tout : les pays qui sont développés économiquement et des les personnes ont confiance en leurs proches seraient aussi assez libre, avec une population en bonne santé ayant facilement accés à des soins, et cela serait lié à moins de corruption dans le pays et amènerait à plus de générosité de la part de la population.